## THỐNG KÊ VÀ PHÂN TÍCH DỮ LIỆU

In [ ]:
import matplotlib.pyplot as plt
import os
import pandas as pd
import seaborn as sns
from PIL import Image
import glob
import random

In [ ]:
PWD_dataset_folder = "../Dataset/Phat_project-3"  # Dien vao duong dan den folder chua dataset

###### **Kiểm tra kích thước ảnh**

In [ ]:
# Lấy danh sách tất cả các tệp ảnh .jpg trong thư mục huấn luyện.
images = glob.glob(f"{PWD_dataset_folder}/train/images/*.jpg")
# Chọn ngẫu nhiên 20 ảnh từ danh sách và lặp qua từng ảnh.
for img in random.sample(images, 20):
  # Mở ảnh bằng Pillow.
  img = Image.open(img)

  # Hiển thị ảnh.
  plt.imshow(img)
  # Đặt tiêu đề biểu đồ hiển thị kích thước ảnh.
  plt.title(f"shape: {img.size}")
  # Tắt trục tọa độ.
  plt.axis('off')
  # Hiển thị biểu đồ.
  plt.show()

###### **Đếm class trong dữ liệu**

In [ ]:
# Danh sách các thư mục chứa tệp nhãn cho tập huấn luyện, kiểm tra và thử nghiệm.
ls_images= [
            f"{PWD_dataset_folder}/train/labels",
            f"{PWD_dataset_folder}/valid/labels",
            f"{PWD_dataset_folder}/test/labels"
            ]
# Khởi tạo một từ điển để lưu trữ số lượng đối tượng cho mỗi lớp ('0', '1', '2') trong mỗi tập ('train', 'val', 'test').
dict_ = {
         "train":{
             "0":0,
             "1":0
         },
         "val": {
             "0":0,
             "1":0
         },
         "test": {
             "0":0,
             "1":0
         }
         }

# hàm count để đếm số lượng đối tượng của mỗi lớp trong các tệp nhãn của một thư mục cụ thể.
def count(folder_path: str, tap: str):
  # Liệt kê tất cả các tệp trong thư mục.
  ls_dir = os.listdir(folder_path)
  # Tạo đường dẫn đầy đủ cho mỗi tệp.
  ls_path = [os.path.join(folder_path, file) for file in ls_dir]
  # Lặp qua từng tệp nhãn.
  for path in ls_path:
    with open(path, "r") as f:
      for line in f:
          # Lấy class_id từ dòng đầu tiên của dòng (chuỗi số nguyên).
          class_id = int(line.split()[0])
          # Tăng số lượng đếm cho class_id tương ứng trong từ điển dict_.
          dict_[f'{tap}'][f'{class_id}'] +=1

# Lặp qua danh sách các thư mục nhãn và gọi hàm count cho mỗi thư mục.
for folder in ls_images:
  if folder == f"{PWD_dataset_folder}/train/labels":
    count(folder, "train")
  elif folder == f"{PWD_dataset_folder}/valid/labels":
    count(folder, "val")
  else:
    count(folder, "test")

In [ ]:
# Tạo bảng DataFrame từ từ điển dict_ để hiển thị số lượng đối tượng cho mỗi lớp trong mỗi tập.
table = pd.DataFrame(dict_, index=['0', '1'])
# Hiển thị bảng DataFrame.
table

In [ ]:
table.to_csv("C:\\Users\\TAN PHAT\\OneDrive\\Desktop\\Yolo-V8\\fix_imbalanced_data\\table.csv", index=True)

###### **Tỷ lệ tổng số ảnh theo train/val/test**

In [ ]:
# Tính tổng số đối tượng trong mỗi tập ('train', 'val', 'test').
sum_= {'train':table['train'].sum(),"val": table['val'].sum(), "test": table['test'].sum()}
# Tạo bảng DataFrame từ tổng số đối tượng.
table_sum = pd.DataFrame(sum_, index=['sum'])
# Hiển thị bảng DataFrame.
table_sum

In [ ]:

# Tính tổng số đối tượng trong mỗi cột.
totals = table.sum()

# Lấy màu từ Seaborn cho biểu đồ pie.
colors = sns.color_palette('pastel')[0:len(totals)]

# Vẽ biểu đồ pie.
plt.figure(figsize=(6,6))
# Vẽ biểu đồ pie với dữ liệu totals, nhãn là tên các cột (tên tập dữ liệu), màu sắc, định dạng phần trăm và góc bắt đầu.
plt.pie(totals, labels=totals.index, colors=colors, autopct='%1.1f%%', startangle=90)
# Đặt tiêu đề cho biểu đồ.
plt.title('Tỷ lệ tổng số ảnh theo train/val/test')
# Hiển thị biểu đồ.
plt.show()

###### **Tỷ lệ phần trăm của từng nhóm**

In [ ]:
# Chuẩn hóa theo cột (tính phần trăm của mỗi lớp trong từng tập).
percent_df = table.div(table.sum(axis=0), axis=1) * 100

# Vẽ biểu đồ cột chồng.
percent_df.T.plot(kind='bar', stacked=True)
# Đặt tiêu đề cho biểu đồ.
plt.title("Tỷ lệ phần trăm của từng nhóm trong mỗi tập")
# Đặt nhãn cho trục y.
plt.ylabel("Phần trăm (%)")
# Đặt nhãn cho trục x.
plt.xlabel("Tập dữ liệu")
# Đặt vị trí chú thích cho biểu đồ.
plt.legend(title="Nhóm", bbox_to_anchor=(1.05, 1), loc='upper left')
# Tự động điều chỉnh bố cục để tránh chồng lấn.
plt.tight_layout()
# Hiển thị biểu đồ.
plt.show()

###### **Tỷ lệ từng hàng theo cột train**

In [ ]:
column = 'train'
values = table[column]

# Tên cho từng phần trong biểu đồ pie (tên lớp).
labels = ['head','helmet']

colors = sns.color_palette('pastel')[0:len(values)]

plt.figure(figsize=(6,6))
plt.pie(values, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
plt.title(f'Tỷ lệ từng hàng theo cột {column}')
plt.show()